In [ ]:
from pyld import jsonld 
import json
import os 


In [ ]:
kind = 'linktest/'
file = 'test1'
loc = 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/'


print(f'{loc}{kind}{file}')


In [ ]:
jsonld.expand(f'{loc}{kind}{file}')

In [ ]:
# (f'{loc}{kind}{file}')['document']

In [ ]:
loader = jsonld.requests_document_loader()

def cpct(link):
    content = loader(link)['document']
    ctx = loader(content['@context'])['document']
    
    
    
    
    return jsonld.expand(content,ctx)

cpct((f'{loc}{kind}{file}'))

In [ ]:
link = f'{loc}{kind}{file}'
content = loader(link)['document']
content0 = loader(link)['document']
ctx0 = loader(content['@context'])['document']
ct2 = loader('https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/_context')['document']
del ct2['@embed']
ctx = {'@context': {
          '@base': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/',
          '@vocab': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/',
          'id': '@id',
          'type': '@type',
          'universe': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/',
          'realm': {'@container': '@id',
                    '@context': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/_context'
                    },
          'realm2': 
            ct2
            # {
            #         /'@context': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/_context'
            #         }
          },
        '@embed': '@always'}

content['@context'] = ctx['@context']
# content['realm']['atmos']['@context'] ='https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/_context'


print(json.dumps(ctx0,indent=4))


In [ ]:
# jsonld.compact(content,ctx)

jsonld.compact(link,ctx0)

In [ ]:
# jsonld.expand('https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmoschem')

In [ ]:
# content0['@context'] = ctx0['@context']
jsonld.compact(content0,ctx0['@context'])

In [ ]:
jsonld.expand(link,ctx0['@context'])


In [ ]:
import re
loader = jsonld.requests_document_loader()

# Recursively expand and resolve @id fields containing URLs
def recursive_expand(jsonld_doc,compact=True,child=False):
    print('!!',jsonld_doc, include_ctx, compact)
    
    # loader = jsonld.requests_default_loader
    # MAKE a loader with caching capabilities
    # # Expand the current document using pyld
    ldfile = loader(jsonld_doc)['document']
    
    expanded_doc = jsonld.expand(jsonld_doc,options={'defaultLoader': loader})
    
    # Recursively check for any @id fields containing URLs
    for i,item in enumerate(expanded_doc):
    
        item = recursive_resolve_ids(item,compact).copy()
        
        # if compact:
            # include_ctx = True
        if not child:
            item = jsonld.compact(item, ldfile['@context'],options={'base': base(jsonld_doc)})
            
        # if include_ctx:
        #     item['@context'] = ldfile['@context']
        
        # manual fix for the context
        expanded_doc[i] = json.loads(re.sub(r'"context"', '"@context"', json.dumps(item)))
        print('---------------')

    return expanded_doc

# Helper function to recursively resolve @id fields in the document
def recursive_resolve_ids(data,compact):
    if isinstance(data, dict):
        # If an @id field with a URL is found, fetch and expand the content
        # do not create an infinte loop if type exists. 
        if not '@type' in data and '@id' in data and data['@id'].startswith("http"):
            expanded_external_data = recursive_expand(data['@id'],compact,True)  # Recursive call to expand 
            print(expanded_external_data)
            # expanded_external_data[0].update(data)
            data = expanded_external_data[0]
            
            # data['context'] = data['@context']
            # merge other data 
            
        for key, value in data.items():
                data[key] = recursive_resolve_ids(value,compact)

    elif isinstance(data, list):
        # Process lists recursively
        data = [recursive_resolve_ids(item,compact) for item in data]
        
    return data

def base(link):
    n = link.count('/')
    return link.rsplit('/',n-3)[0]

In [ ]:
recursive_expand(link,compact=True)

In [ ]:
rexp = recursive_expand(link,compact=False)
jsonld.compact(rexp,rexp[0]['@context'])

In [ ]:
jsonld.to_rdf(rexp)

dklja


In [ ]:


import json
import re
from typing import Any, Dict, List, Union
from functools import lru_cache
from pyld import jsonld

class JsonLdProcessor:
    """
    A class for processing JSON-LD documents with recursive expansion and ID resolution.
    
    Features:
    - Recursively expands and resolves @id fields containing URLs
    - Handles document caching
    - Supports compact and expanded document formats
    - Maintains context handling
    """
    
    def __init__(self):
        """Initialize the processor with a cached document loader."""
        self.loader = jsonld.requests_document_loader()
    
    @staticmethod
    def _extract_base_url(url: str) -> str:
        """
        Extract the base URL from a full URL path.
        
        Args:
            url: The full URL to process
            
        Returns:
            The base URL containing protocol and domain
        """
        parts = url.split('/')
        return '/'.join(parts[:3])
    
    @lru_cache(maxsize=100)
    def _load_document(self, url: str) -> Dict:
        """
        Load and cache a JSON-LD document from a URL.
        
        Args:
            url: The URL to fetch the document from
            
        Returns:
            The loaded document
        """
        return self.loader(url)['document']
    
    def _resolve_ids(self, 
                     data: Union[Dict, List], 
                     compact: bool = True) -> Union[Dict, List]:
        """
        Recursively resolve @id fields in the document.
        
        Args:
            data: The data structure to process
            include_ctx: Whether to include context in the result
            compact: Whether to compact the resulting document
            
        Returns:
            The processed data structure with resolved IDs
        """
        if isinstance(data, dict):
            # Only process @id fields that don't have a @type and start with http
            if '@id' in data and not '@type' in data and data['@id'].startswith('http'):
                expanded = self.expand_document(
                    data['@id'],
                    compact=compact,
                    is_nested=True
                )
                if expanded:
                    data = expanded[0]
            
            # Process all dictionary values recursively
            return {
                key: self._resolve_ids(value, compact)
                for key, value in data.items()
            }
        
        elif isinstance(data, list):
            return [self._resolve_ids(item, compact) for item in data]
        
        return data
    
    def expand_document(self,
                       jsonld_doc: Union[str, Dict],
                       compact: bool = True,
                       expand_ctx: bool = True,
                       expand_links: bool = True,
                       is_nested: bool = False) -> List[Dict]:
        """
        Expand a JSON-LD document and resolve all referenced URLs.
        
        Args:
            jsonld_doc: The JSON-LD document to process (URL or dict)
            include_ctx: Whether to include context in the result
            compact: Whether to compact the final document
            is_nested: Whether this is a nested expansion
            
        Returns:
            List of processed documents
        """
        # Load and expand the document
        doc = self._load_document(jsonld_doc) if isinstance(jsonld_doc, str) else jsonld_doc
        expanded = jsonld.expand(jsonld_doc, options={'defaultLoader': self.loader})
        
        # Process each item in the expanded document
        processed = []
        for item in expanded:
            if expand_links:
                processed_item = self._resolve_ids(item, compact).copy()
            else:
                processed_item = item.copy()
            
            # Handle compaction for root-level documents
            if compact and not is_nested:
                base_url = self._extract_base_url(jsonld_doc) if isinstance(jsonld_doc, str) else None
                processed_item = jsonld.compact(
                    processed_item,
                    doc['@context'],
                    options={'base': base_url} if base_url else {}
                )
            
            # Fix context notation if needed
            if isinstance(processed_item, dict):
                processed_item = json.loads(
                    re.sub(r'"context"', '"@context"', json.dumps(processed_item))
                )
            
            if expand_ctx and '@context' in processed_item and isinstance(processed_item['@context'], str):
                processed_item['@context'] = self.loader(processed_item['@context'])['document']
                
                
            processed.append(processed_item)
            
            
        
        return processed

In [ ]:
import cmipld

processor = cmipld.JsonLdProcessor()

In [ ]:

result = processor.expand_document(link,compact=True,expand_ctx=False)
print(json.dumps(result,indent=4))

In [ ]:
result = processor.expand_document(link,compact=True,expand_ctx=False,expand_links=False)
print(json.dumps(result,indent=4))

In [ ]:
import pprint
pprint.pprint(jsonld.compact(result,result[0]['@context']))

In [ ]:
jsonld.compact(link,{'@context': {"_": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/"}})

In [ ]:
from urllib.parse import urljoin

def depends(link,relative=False):
    
    ids = jsonld.frame(link,{'@explicit': True})['@graph']
    if relative:
        return set([i['@id'] for i in ids])
    else:
        return set([urljoin(link,i['@id']) for i in ids])

depends(link)

In [ ]:
def simple_load(link):
    # basic compact
    data = loader(link)['document']
    
    ctx = loader(data['@context'])['document']
    
    print(ctx)
    
    return jsonld.compact(link,ctx)

simple_load(link)

In [ ]:
jsonld.to_rdf(link)

In [ ]:
jsonld.compact('https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/graph.jsonld','https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/_context')


In [ ]:
jsonld.frame('https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/graph.jsonld',{'@graph':{}})

In [ ]:
jsonld.expand('https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/graph.jsonld',{'@graph':{}})

In [ ]:
# script groups jsonld files into a single file
# jsonld.compact('https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/graph.min.jsonld','https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/_context')
# graphs files should not have multiple contexts. (only one overarching one.)


# broken link checker. 
# JSLD checker. 

In [ ]:
jsonld.compact('https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/test2',['https://wcrp-cmip.github.io/WCRP-UNIVERSE/linktest/_context','https://wcrp-cmip.github.io/WCRP-UNIVERSE/_globalcontext'])




In [ ]:
# jsonld.expand('https://wcrp-cmip.github.io/WCRP-UNIVERSE/graph.jsonld')

updatejson

In [ ]:
from cmipld import JsonLdProcessor
processor = JsonLdProcessor()
data = processor.get('https://raw.githubusercontent.com/WCRP-CMIP/CMIP6Plus_CVs/refs/heads/production/data_descriptors/experiment/abrupt_4xco2.json')[0]

del data['@context']
data['te'] = {'aer': {},
  'aogcm': {},
  'bgc': {},
  'chem': {}}

data

In [ ]:
jsonld.compact('https://wcrp-cmip.github.io/CMIP6Plus_CVs/project_definition.json','https://wcrp-cmip.github.io/CMIP6Plus_CVs/_context')

In [ ]:
from cmipld import JsonLdProcessor
processor = JsonLdProcessor()
processor.expand_document('https://wcrp-cmip.github.io/CMIP6Plus_CVs/project_definition.json',compact=True,expand_ctx=True,no_ctx=False,pprint=True)

In [ ]:
from pyld import jsonld

In [ ]:
jsonld.expand('https://WCRP-CMIP.github.io/WCRP-UNIVERSE/graph')

In [ ]:
processor.expand_document('https://WCRP-CMIP.github.io/WCRP-UNIVERSE/graph.jsonld',compact=False,expand_ctx=True,no_ctx=False,pprint=True)

In [ ]:
jsonld.compact('https://WCRP-CMIP.github.io/WCRP-UNIVERSE/mip/graph.jsonld','https://WCRP-CMIP.github.io/cmip6plus_cvs/_context')

In [ ]:
jsonld.expand('https://wcrp-cmip.github.io/CMIP6Plus_CVs/source/giss_e2_1_g.json') #    ,{})['@graph'][0]

In [ ]:
jsonld.frame('https://wcrp-cmip.github.io/WCRP-UNIVERSE/mip/cmip6.json',"https://wcrp-cmip.github.io/WCRP-UNIVERSE/mip/cmip6.json")

In [ ]:
jsonld.compact('https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmos.json','https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/_context_')

In [ ]:
import cmipld

processor = cmipld.JsonLdProcessor()

In [ ]:
doc = processor._load_document('https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmos.json')
doc

In [ ]:
processor.expand_document('https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/graph.jsonld',compact=False,expand_ctx=True,expand_links=False)

In [ ]:
jsonld.expand('https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/graph.jsonld')

In [ ]:
# jsonld.frame({
#   "@context": {
#   "@base": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/",
#   "@vocab": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/",
#   "id": "@id",
#   "type": "@type",
#   "cf": "https:/wcrp-cmip.github.io/WCRP-CMIP/CF/",
#   "cmip6plus": "https:/wcrp-cmip.github.io/CMIP6Plus_CVs/",
#   "cmip7": "https:/wcrp-cmip.github.io/CMIP7_CVs/",
#   "mip-variables": "https://wcrp-cmip.github.io/MIPvariables/",
#   "wcrp-universe": "https:/wcrp-cmip.github.io/WCRP-UNIVERSE/"
# },

#   "@graph": [
#     {
#   "description": "Aerosol",
#   "id": "aerosol",
#   "name": "aerosol",
#   "type": "realm"
# }
#     ,{
#   "description": "Atmosphere",
#   "id": "atmos",
#   "name": "atmos",
#   "type": "realm"
# }
#   ]},
# {  "@context": {
#   "@base": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/",
#   "@vocab": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/",
#     "id": "@id",
#   "type": "@type",
#   },
#   '@id': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmos',"@embed": "@always"})

In [ ]:
inputDocument = {
  "@context": {
    "@base": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/",
    "@vocab": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/",
    "id": "@id",
    "type": "@type",
    "cf": "https:/wcrp-cmip.github.io/WCRP-CMIP/CF/",
    "cmip6plus": "https:/wcrp-cmip.github.io/CMIP6Plus_CVs/",
    "cmip7": "https:/wcrp-cmip.github.io/CMIP7_CVs/",
    "mip-variables": "https://wcrp-cmip.github.io/MIPvariables/",
    "wcrp-universe": "https:/wcrp-cmip.github.io/WCRP-UNIVERSE/"
  },
  "@graph": [
    {
      "description": "Aerosol",
      "id": "aerosol",
      "name": "aerosol",
      "type": "realm"
    },
    {
      "description": "Atmosphere",
      "@id": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmos",
      "name": "atmos",
      "type": "realm"
    }
  ]
}

frame = {
  "@context": {
    "@base": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/",
    "@vocab": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/"
  },
  "@id": "https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmos", 
  "@embed": "@always",
  "@explicit": False
}



jsonld.expand(inputDocument)



In [ ]:
jsonld.frame(inputDocument, frame)

In [109]:
jsonld.expand('https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/graph.jsonld')

JsonLdError: ('Could not retrieve a JSON-LD document from the URL.',)
Type: jsonld.LoadDocumentError
Code: loading document failed
Cause: Expecting value: line 1 column 1 (char 0)  File "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/pyld/documentloader/requests.py", line 72, in loader
    'document': response.json()
  File "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/requests/models.py", line 978, in json
    raise RequestsJSONDecodeError(e.msg, e.doc, e.pos)
